# *Learning Redundancy in Supply Chain Networks: Analysis of Topological and Structural Signals*

**Goal**: Calculate and predict *real* supply chain redundancy. Comparing the use of structural metrics vs using topological metrics vs using both metrics during learning. 

**Motivation**: There is a problem of the problem with the *illusion* of redundancy in supply chain networks. Supply chain audits rely on degree, number of suppliers, number of routes, but these fail to detect shared failure modes, meaning that "redundant" edges or nodes may not be redundant at all.

**Structural Metrics:** Quantify local geometry and path-based properties of the graph. These metrics are typically sensitive to degrees, distances, and flow-like behavior, but do not explicitly encode higher-order connectivity.
- Degree change: Measures whether removal of X locally alters node connectivity.
- Shortest-path distortion: Captures whether alternative routes preserve path efficiency.
- Edge / node betweenness change: Measures whether traffic or flow is re-routed through different parts of the network after removal.
- Local efficiency: Quantifies robustness of information exchange in the neighborhood.
- Spectral stability: Measures sensitivity of the graph Laplacian spectrum to removal.

**Topological Metrics:** Quantify higher-order connectivity and redundancy of paths, capturing global structural features that persist across scales. Initial idea is that topological redundancy reflects whether X participates in persistent alternative structures rather than merely short local detours.
- Connected components (0-dimensional homology): Detects whether removal disconnects the graph.
- Cycle persistence (1-dimensional homology): Measures destruction of cycles that encode alternative pathways.
- Total persistence: Captures how much “redundancy mass” is lost.
- Loop stability across filtrations: Evaluates whether cycles persist under multiple edge-weight or distance filtrations.

---

## *Defining Redundancy of a Supply Chain Route*

### What is Route/Edge Redundancy?

**In general:** Let X be an edge or set of connected edges. Remove X from G to produce new graph G\X. X is redundant if change between G and G\X is negligible under a chosen family of structural and topological metrics.

**In a Supply Chain Network:** Let X be an edge or set of connected edges. Remove X from G to produce new graph G\X. X is redundant if the removal of X causes only negligible impact on the network’s ability to deliver materials or products to all nodes. More concretely, X is redundant if:
- Reachability is largely preserved: Most or all nodes can still be reached from their sources after removing X
- Delivery efficiency is minimally affected: The time, distance, or cost required to move products through the network changes only slightly for the nodes that remain reachable.

In other words, redundancy measures whether X participates in critical pathways or whether alternative routes and structures exist that allow the supply chain to function effectively even in its absence. 

### Creating a Redundancy Score for any X

With those definitions in mind, we will define a set of edges \(X\)'s *redundancy score* as:
$$
R(X) = 
\sum_{v \in V_{\text{reachable}}} \big( t_{G \setminus X}(v) - t_G(v) \big) 
+ \lambda \cdot |V_{\text{unreachable}}|
$$

where:  

- $V_{\text{reachable}}$ is the set of nodes still reachable after removing $X$,  
- $V_{\text{unreachable}}$ is the set of nodes that can no longer be reached,  
- $t_G(v)$ is the time (or number of weighted steps) it takes for node $v$ to receive material in the original graph $G$,  
- $t_{G \setminus X}(v)$ is the corresponding time in the modified graph $G \setminus X$,  
- $\lambda$ is a large (and negative) penalty weight for unreachable nodes. Large because unreachable nodes should dominate delay.

The greater the redundancy score $R(X)$, the more redundant $X$ is and vice versa.


---
## *Computing the Redundancy Score*

### Reducing Complexity of Computing the Score

Computing $R(X)$ exactly for every candidate removal $X$ during training can be expensive (we would be recomputing a diffusion or flow simulation per $X$). We can instead approximate $R(X)$ using diffusion and the heat equation:

- Model diffusion by the heat equation on the graph Laplacian $L$. Start from an initial source signal $f(0)$ (e.g., supply injection at source nodes). The diffusion at time $t$ is $ f(t) = e^{-tL} f(0) $

- Define the arrival time of node $v$ in graph $G$ as the first time its diffused mass crosses a threshold $\tau$. That is, let $t_G(v) = \min\{ t \ge 0 : f_v(t) \ge \tau \}$. 

- Then the redundancy score is $R(X) = \sum_{v \in V_{\text{reachable}}} \big(t_{G\setminus X}(v) - t_G(v)\big) + \lambda \cdot |V_{\text{unreachable}}|$

To do spectral (fast) approximation, we compute just the top $k$ eigenpairs $(\lambda_i, \phi_i)$ of $L$ (once), and use the truncated heat kernel $e^{-tL} \approx \sum_{i=1}^k e^{-t\lambda_i} \phi_i \phi_i^\top$. When an edge set $X$ is removed, the Laplacian changes by a sparse perturbation $\Delta L$. Use first-order perturbation to update eigenvalues $\tilde\lambda_i \approx \lambda_i + \phi_i^\top (\Delta L) \phi_i$ and recompute the truncated sum with $\tilde\lambda_i$. Arrival times $t_{G\setminus X}(v)$ are then derived from the approximated $f_{G\setminus X}(t)$.


### Choice of Diffusion for Redundancy Metric

1. Single vs Multiple Sources? 

   Multiple sources. Most supply chains have several suppliers or warehouses feeding the network simultaneously. Modeling diffusion from multiple sources better captures real-world redundancy and alternative routing.

2. Linear vs Nonlinear Congestion? 

   Linear flow approximation. Assume that the “flow” of material is proportional to edge capacities and additive. Nonlinear congestion (e.g., queueing, bottlenecks) could be modeled in the future, but linear flow keeps computation tractable for large graphs.

3. Steady-state vs Transient Dynamics?  

   Transient dynamics. Measure how long it takes nodes to receive material after removing $X$, rather than just the steady-state distribution. This aligns with our delay-based redundancy score $R(X)$ and captures the temporal fragility of the network.

Note that our $R(X)$ metric is highly dependent on the diffusion model we choose! Future work could involve changing the diffusion choice used.